In [14]:
import numpy as np;
import pandas as pd;
import plotly.express as px;


In [15]:
# Global parameters
NO_OF_DAYS = 200
modelParameter = {}
modelParameter["beta"] = 0.49  
modelParameter["tE"] = 5
modelParameter["tI"] = 5
modelParameter["tH"] = 9
modelParameter["mu"] = 0.001

print(modelParameter)

# System variables
ROOT_DIR = "/Users/sreeramh/Documents/Kerala_Covid/SpatioTemporal/gitsrc/SpatioTemporalSEIR"

{'beta': 0.49, 'tE': 5, 'tI': 5, 'tH': 9, 'mu': 0.001}


In [22]:
# Load data
initDataDF = pd.read_csv(ROOT_DIR + "/data/tvm_init_data.csv")
#print(initDataDF)

cSparseMatrix = pd.read_csv(ROOT_DIR + "/output/cmatrix_results.csv")
#print(neighborDF)


In [23]:
## Differential equation
def deriv(y, Ni, contactRatio, modelParameter):
    Si, Ei, Ii, Hi, Ri = y
    dSdt = -modelParameter["beta"] * Si * contactRatio
    dEdt = (modelParameter["beta"] * Si * contactRatio) - (Ei/modelParameter["tE"])
    dIdt = (Ei/modelParameter["tE"]) -  (Ii/modelParameter["tI"])
    dHdt = (Ii/modelParameter["tI"]) - (Hi/modelParameter["tH"])
    dRdt = (Hi/modelParameter["tH"])
    return dSdt, dEdt, dIdt, dHdt, dRdt

In [24]:
def findSpatialContactRatio(name, NDict, yDict, cSparseMatrix):
    # TODO conver this to a matrix operation
    contactRatio = 0;
    Ci = cSparseMatrix[cSparseMatrix['iName']==name]
    for index, Cij in Ci.iterrows():
        jName = Cij['jName']
        cij = Cij['Cij']
        Ij = yDict[jName][2]
        Nj = NDict[jName]
        contactRatio = contactRatio +  (cij*Ij/Nj)
    return contactRatio;
        

In [25]:
## next count 
def derivNext(y, rates) :
    S,E,I,H, R = y
    dSdt, dEdt, dIdt, dHdt, dRdt = rates
    S1 = S + dSdt
    E1 = E + dEdt
    I1 = I + dIdt
    H1 = H + dHdt
    R1 = R + dRdt
    return S1, E1, I1, H1, R1


In [26]:
yDict = {};
NDict = {};
resultsDF = pd.DataFrame(columns=['name', 'day', 'S', 'E', 'I', 'H', 'R'], );

for index, row in initDataDF.iterrows():
    NDict[row['name']] = row['S'] + row['E'] + row['I'] + row['H'] + row['R']
    yDict[row['name']] = row['S'], row['E'], row['I'], row['H'], row['R'] 
    

for day in range(1,NO_OF_DAYS):
    for index, row in initDataDF.iterrows():
        name = row['name'];
        y = yDict[name]; 
        
        contactRatio = findSpatialContactRatio(name, NDict, yDict, cSparseMatrix);
        #print(name, "contactRatio = ", contactRatio)
        
        rates = deriv(y, NDict[name], contactRatio, modelParameter)
        yNext = derivNext(y, rates) 
        
        yDict[name] = yNext
        #print(name, yNext)
        resultsDF = resultsDF.append({'name': name, 'day': day, 'S': yNext[0], 'E': yNext[1], 'I': yNext[2], 'H': yNext[3], 'R': yNext[4]},ignore_index=True)


In [27]:
resultsDF.head(10)

,name,day,S,E,I,H,R
0,Amboori,1,15919.999955,0.000045,0.0,0.0,0.0
1,Anad,1,31686.999750,0.000250,0.0,0.0,0.0
2,Anchuthengu,1,17395.999955,0.000045,0.0,0.0,0.0
3,Andoorkonam,1,30780.999677,0.000323,0.0,0.0,0.0
4,Aruvikkara,1,33395.999447,0.000553,0.0,0.0,0.0
5,Aryanad,1,26360.999875,0.000125,0.0,0.0,0.0
6,Aryancodu,1,24327.999900,0.000100,0.0,0.0,0.0
7,Athiyannoor,1,26972.999847,0.000153,0.0,0.0,0.0
8,Attingal(M),1,37435.999874,0.000126,0.0,0.0,0.0
9,Azhoor,1,27389.999861,0.000139,0.0,0.0,0.0


In [28]:
resultsDF.to_csv(ROOT_DIR + "/output/results.csv")

In [29]:
fig = px.scatter(resultsDF, x="day", y="H", color="name")
fig.show()